In [1]:
CONTRACT_ADDRESS = "0x52e55a93a1fda4f8394f1090c2b52bd4c5c92b69"
COLLECTION_NAME = "thesaudiokaybears"
COMPRESS_RAW_DATA = False
SAVE_RAW_DATA = False

NFTNERDS_BASE_URL = "https://d1h1fvurhxrz4j.cloudfront.net/metadata/"

In [2]:
"""
@date: 11/01/2022
"""

# Collection name will be collected from raritysniffer response

import os
import requests
import time
import datetime
import json
import shutil
import pandas as pd

from honestnft_utils import config

dt_now = datetime.datetime.utcnow()
start_time = time.time()

print(f"Working in directory: {os.getcwd()}")

url = f"{NFTNERDS_BASE_URL}{CONTRACT_ADDRESS}"

headers = {
    "Content-Type": "application/json",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36",
}

response = requests.request("GET", url, headers=headers)

if response.status_code == 200:
    response_data = response.json()

    # list to make completeness checks
    token_ids = []

    # TODO
    # - handle collections with no data

    # Create folder to store metadata
    folder = f"{config.ATTRIBUTES_FOLDER}/{COLLECTION_NAME}/"
    print(f"Saving raw attributes to {folder}")
    if not os.path.exists(folder):
        os.mkdir(folder)

    for token in response_data["traitsDict"]:
        token_dict = dict()
        token_ids.append(int(token))
        token_data = response_data["traitsDict"][token]
        token_dict["tokenId"] = token
        token_dict["attributes"] = response_data["traitsDict"][token]
        PATH = f"{config.ATTRIBUTES_FOLDER}/{COLLECTION_NAME}/{token}.json"

        with open(PATH, "w") as destination_file:
            json.dump(token_dict, destination_file)

    print(f"tokens in the list {len(token_ids)}")
    print(f"lower_id: {min(token_ids)}")
    print(f"upper_id: {max(token_ids)}")

    if min(token_ids) == 0:
        max_supply = max(token_ids) + 1
        print(f"max supply: {max(token_ids) + 1}")
    else:
        max_supply = max(token_ids)
        print(f"max supply: {max(token_ids)}")

    # warn if nftNerds doesn't have the full collection
    # TODO: print a list of missing token ids
    if len(token_ids) != max_supply:
        print(f"{max_supply - len(token_ids)} tokens with missing metadata")

    # Run pulling.py to grab missing tokens and create rarity csv
    print(
        f"Creating rarity csv with 'pulling.py' using the command: python3 pulling.py --contract {CONTRACT_ADDRESS} --collection {COLLECTION_NAME}"
    )
    os.system(
        f"python3 pulling.py --contract {CONTRACT_ADDRESS} --collection {COLLECTION_NAME}"
    )

    # Calculate rarity_data csv with rarity.py
    print(
        f"Calculating rarity data with 'rarity.py' using the command: python3 rarity.py --collection {COLLECTION_NAME}"
    )
    os.system(f"python3 rarity.py --collection {COLLECTION_NAME}")

    # Compress raw data and delete folder
    if COMPRESS_RAW_DATA:
        print("Compressing raw metadata")
        dir_name = f"{config.ATTRIBUTES_FOLDER}/{COLLECTION_NAME}"
        shutil.make_archive(dir_name, "zip", dir_name)
        shutil.rmtree(dir_name)

    print("--- %s seconds to process collection" % (round(time.time() - start_time, 1)))

else:
    print(f"Received a {response.status_code} error from the NFT Nerds API")

print("finished.")

Working in directory: /home/flyingfish/honestnft-shenanigans/metadata
Saving raw attributes to /home/flyingfish/honestnft-shenanigans/data/raw_attributes/thesaudiokaybears/
tokens in the list 2222
lower_id: 0
upper_id: 2221
max supply: 2222
Calculating rarity data with 'rarity.py' using the command: python3 pulling.py --contract 0x52e55a93a1fda4f8394f1090c2b52bd4c5c92b69 --collection thesaudiokaybears
Metadata for lower bound token id is at: https://nftservices.s3.amazonaws.com/thesaudiokaybears/0.json
Fetching data for thesaudiokaybears
Lower ID: 0
Upper ID: 2221
Max supply: 2222
         TOKEN_NAME Background Fur Clothes Eyes Head Mouth Glasses
TOKEN_ID                                                          
0           UNKNOWN      Cream   4       1    5   10    37       0
1           UNKNOWN       Blue   1       5    4    0    37       5
2           UNKNOWN      Green   4       3    6    8    18       0
3           UNKNOWN      Cream   8       2    6    1    18       2
4         